In [321]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# --------------------------------------------Data Cleaning & Preprocessing--------------------------------------------------------

# Load the dataset
df = pd.read_csv("/Users/jz/Desktop/ML Final/class_final_proportion.csv")

# Drop rows with missing DV values
df1 = df[df['cabinet_proportion'].notna()]

# Drop columns because they're string vars or it's not quite interpretable to make a dummy var for every id 
to_drop = [
    'party', 'base', 'country', 'cabinet_id', 'party_id', 'cabinet_name', 
    'party_name', 'party_name_english', 'country_id', 'election_id', 
    'election_date', 'start_date', 'post_election', 'lag_largest_parl', 
    'lag_largest_cab'
]
df_semi_dropped = df1.drop(columns=to_drop)

# Identify columns with more than 15 missing values and drop them
missing_values = df_semi_dropped.isna().sum()
to_drop = missing_values[missing_values > 15].index
df_dropped_initial = df_semi_dropped.drop(columns=to_drop)

# **Convert Binary Columns to Numeric Before Calculating Mean and Std**
binary_columns_initial = [
    col for col in df_dropped_initial.columns 
    if df_dropped_initial[col].dropna().astype(str).isin(['0', '1']).all()
]
for col in binary_columns_initial:
    df_dropped_initial[col] = df_dropped_initial[col].astype(int)

# **Calculate Mean and Std Before Removing Missing Entries and Outliers**
# Select only numeric columns for mean and std calculation
numeric_columns = df_dropped_initial.select_dtypes(include=[np.number]).columns
mean_before = df_dropped_initial[numeric_columns].mean()
std_before = df_dropped_initial[numeric_columns].std()

# **Remove rows with any remaining missing values**
df_dropped_no_na = df_dropped_initial.dropna()

# Function to remove rows with outliers
def remove_outliers(df, threshold=10):
    # Identify binary columns and convert to integers (if not already)
    binary_columns = [
        col for col in df.columns 
        if df[col].dropna().astype(str).isin(['0', '1']).all()
    ]
    for col in binary_columns:
        df[col] = df[col].astype(int)
    
    # Re-select numeric columns after conversion
    numeric_df = df.select_dtypes(include='number')
    
    # Calculate mean and standard deviation for numeric columns
    means = numeric_df.mean()
    stds = numeric_df.std()
    
    # Create a mask for rows that are within the threshold for all numeric columns
    mask = (numeric_df - means).abs() <= (threshold * stds)
    valid_rows = mask.all(axis=1)
    
    # Return the cleaned DataFrame (retain all original columns)
    return df[valid_rows]

# **Remove Outliers**
df_dropped_cleaned = remove_outliers(df_dropped_no_na, threshold=10)

# **Calculate Mean and Std After Removing Missing Entries and Outliers**
mean_after = df_dropped_cleaned[numeric_columns].mean()
std_after = df_dropped_cleaned[numeric_columns].std()

# **Compute the Change in Mean and Std**
delta_mean = mean_after - mean_before
delta_std = std_after - std_before

# **Create a Summary DataFrame of Changes**
stats_changes = pd.DataFrame({
    'Mean_Before_Cleaning': mean_before,
    'Mean_After_Cleaning': mean_after,
    'Delta_Mean': delta_mean,
    'Std_Before_Cleaning': std_before,
    'Std_After_Cleaning': std_after,
    'Delta_Std': delta_std
})


# **Print the Summary of Changes**
print("Changes in Mean and Standard Deviation Before and After Cleaning:\n")
print(stats_changes)

# --------------------------------------------Standardization and Final Output--------------------------------------------------------

# Define binary variables
binary_variables = [
    'sq_cabinet', 'sq_pm', 'caretaker', 'cabinet_party', 'prime_minister', 'mingov', 
    'bicameral', 'largest_parl', 'largest_cab', 
    'A', 'B', 'B_star', 'C', 'D', 'E', 
    'country_dummy1', 'country_dummy2', 'country_dummy3', 'country_dummy4', 
    'country_dummy5', 'country_dummy6', 'country_dummy7', 'country_dummy8', 
    'country_dummy9', 'country_dummy10', 'country_dummy11', 'country_dummy12', 
    'country_dummy13'
]

# **Ensure that all binary variables exist in the cleaned DataFrame**
existing_binary_vars = [var for var in binary_variables if var in df_dropped_cleaned.columns]

# **Separate the binary and non-binary data**
binary_data = df_dropped_cleaned[existing_binary_vars]
non_binary_data = df_dropped_cleaned.drop(columns=existing_binary_vars)

# **Additions Start: Exclude 'cabinet_proportion' from standardization**
# Extract 'cabinet_proportion' to exclude it from standardization
cabinet_proportion = non_binary_data['cabinet_proportion']
# Drop 'cabinet_proportion' from non_binary_data
non_binary_data = non_binary_data.drop(columns=['cabinet_proportion'])
# **Additions End**

# **Standardize the non-binary data**
scaler = StandardScaler()
standardized_array = scaler.fit_transform(non_binary_data)
standardized_data = pd.DataFrame(standardized_array, columns=non_binary_data.columns)

# **Additions Start: Reattach 'cabinet_proportion' after standardization**
# Add 'cabinet_proportion' back to the standardized data
standardized_data['cabinet_proportion'] = cabinet_proportion.reset_index(drop=True)
# **Additions End**

# **Reset the index of both DataFrames to ensure alignment**
standardized_data.reset_index(drop=True, inplace=True)
binary_data.reset_index(drop=True, inplace=True)

# **Now concatenate**
df_dropped = pd.concat([standardized_data, binary_data], axis=1)

# **Optional:** Verify the final row count and alignment
print(f"\nFinal dataset shape: {df_dropped.shape}")  # Corrected variable from df_final to df_dropped

# Save the final standardized and combined dataset to CSV
df_dropped.to_csv('/Users/jz/Desktop/ML Final/output.csv', index=False)


Changes in Mean and Standard Deviation Before and After Cleaning:

                    Mean_Before_Cleaning  Mean_After_Cleaning  Delta_Mean  \
seats                          32.100457            31.936137   -0.164320   
sq_pm                           0.128882             0.127726   -0.001156   
election_year                1998.307458          1998.372274    0.064816   
miw_new                         8.864536             8.247664   -0.616872   
banzhaf                         0.129328             0.130372    0.001044   
shapley                         0.129337             0.130400    0.001063   
splus                           0.129289             0.130466    0.001177   
caretaker                     -15.158295             0.062305   15.220601   
cabinet_party                   0.350076             0.350467    0.000391   
prime_minister                  0.127854             0.129283    0.001430   
cabinet_seats                   2.304414             2.302181   -0.002233   
total_cab

/var/folders/z6/93vk8zs14dn7td0ly8dm9vz00000gn/T/ipykernel_80030/1371269647.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int)
/var/folders/z6/93vk8zs14dn7td0ly8dm9vz00000gn/T/ipykernel_80030/1371269647.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int)
/var/folders/z6/93vk8zs14dn7td0ly8dm9vz00000gn/T/ipykernel_80030/1371269647.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [326]:
# -------------------------------------------------------PCA-------------------------------------------------------------

from sklearn.decomposition import PCA

def myPCA(merge_vars, merge_dim, data_frame, latent_description):
    """
    Perform PCA on specified variables and return the PCA object along with the updated DataFrame.

    Parameters:
    - merge_vars: List of variable names to be merged.
    - merge_dim: Number of latent variables after merge.
    - data_frame: The DataFrame containing the data.

    Returns:
    - pca: Fitted PCA object.
    - latent_df: DataFrame with only the PCA variables.
    
    """
    # Ensure all merge_vars exist in the DataFrame
    missing_vars = [var for var in merge_vars if var not in data_frame.columns]
    if missing_vars:
        raise KeyError(f"The following columns are missing in the DataFrame: {missing_vars}")

    # Extract data for PCA
    data = data_frame[merge_vars].values

    # Perform PCA
    pca = PCA(n_components=merge_dim)
    latent_vars = pca.fit_transform(data)

    # Create a DataFrame for latent variables
    latent_df = pd.DataFrame(latent_vars, columns=[f'latent_var_{i+1} for {latent_description}' for i in range(merge_dim)], index=data_frame.index)


    return pca, latent_df, data_frame


import math

def top_variables_by_latent(pca, variables, total_vars): #works as interpretor
    """
    Given a PCA object and a list of original variable names, 
    returns a formatted string of the top original variables 
    and the variance explained for each variable in each latent variable (principal component).
    
    Parameters:
    - pca: the fitted PCA object
    - variables: list of original variable names used in PCA
    - total_vars: total number of original variables
    
    Returns:
    - A formatted string listing the top variables with their variance explained for each principal component.
    """
    # Initialize a list to store formatted strings for each component
    result = []
    
    # Calculate the threshold for selecting top variables based on explained variance
    threshold_fraction = 1 / total_vars
    
    # Create a DataFrame of the absolute loadings for easier manipulation
    loadings = pd.DataFrame(np.abs(pca.components_), columns=variables)
    
    # Loop over each principal component
    for i, explained_variance in enumerate(pca.explained_variance_ratio_):
        # Calculate the number of top variables to select based on explained variance
        num_top_vars = math.ceil(explained_variance / threshold_fraction)
        
        # Get the top variables for this principal component by sorting loadings
        component_loadings = loadings.iloc[i]
        top_variables = component_loadings.nlargest(num_top_vars + 2).index.tolist()
        
        # Format each top variable with its variance explained in the current component
        variable_info = []
        for var in top_variables:
            var_loading = component_loadings[var]
            var_variance_explained = var_loading  # Approximate variance explained
            variable_info.append(f"{var} ({var_variance_explained:.2%})")
        
        # Format the result for this component
        component_str = f"PC{i+1} ({explained_variance:.2%} total variance explained): " + ", ".join(variable_info)
        result.append(component_str)
    
    # Join all component strings with new lines for a final formatted output
    return "\n\n".join(result)
    
# Example usage:
# formatted_string = top_variables_by_latent(pca2, merge_var, len(merge_var))
# print(formatted_string)
df_final.to_csv('/Users/jz/Desktop/ML Final/output.csv', index=False)

In [328]:
# Define the variables to be merged and perform PCA1

merge_var = ['seats', 'cabinet_seats', 'total_cabinet_size','seats_share', 
             'seats_total', 'seats_proportion']

pca2, new_df, df_dropped = myPCA(merge_var, 2, df_dropped, 'population_based_party_influence')

threshold = 1 / len(merge_var)
print("Variance explained by each latent variable in PCA: ", pca2.explained_variance_ratio_,
      pca2.explained_variance_ratio_.cumsum(), f'in {threshold} threshold for loadings')

# Calculate the absolute loadings of each component for each variable
loadings = pd.DataFrame(
    np.abs(pca2.components_),
    columns=merge_var,
    index=[f'PC{i+1}' for i in range(pca2.n_components_)]
)

# Identify variables to drop based on the threshold
vars_to_drop = loadings.columns[(loadings >= threshold).any(axis=0)]
print("Variables to be dropped based on loadings threshold:", vars_to_drop.tolist())

# Drop only those identified variables from df_dropped
df_dropped = df_dropped.drop(columns=vars_to_drop)
loadings

Variance explained by each latent variable in PCA:  [0.5251645  0.28602597] [0.5251645  0.81119047] in 0.16666666666666666 threshold for loadings
Variables to be dropped based on loadings threshold: ['seats', 'cabinet_seats', 'total_cabinet_size', 'seats_share', 'seats_total', 'seats_proportion']


,seats,cabinet_seats,total_cabinet_size,seats_share,seats_total,seats_proportion
PC1,0.460831,0.447505,0.040247,0.537872,0.051398,0.542039
PC2,0.278169,0.009861,0.637976,0.129442,0.696205,0.118552


In [330]:
# Define the variables to be merged and perform PCA2

merge_var = ['miw_new', 'banzhaf', 'shapley','splus', 
             'party_count', 'cab_count','enpp','miw_proportion','W']

pca2, new_df2, df_dropped = myPCA(merge_var, 3, df_dropped, 'party_pivitality')

threshold = 1 / len(merge_var)
print("Variance explained by each latent variable in PCA: ", pca2.explained_variance_ratio_,
      pca2.explained_variance_ratio_.cumsum(), f'in {threshold} threshold for loadings')

# Calculate the absolute loadings of each component for each variable
loadings = pd.DataFrame(
    np.abs(pca2.components_),
    columns=merge_var,
    index=[f'PC{i+1}' for i in range(pca2.n_components_)]
)

# Identify variables to drop based on the threshold
vars_to_drop = loadings.columns[(loadings >= threshold).any(axis=0)]
print("Variables to be dropped based on loadings threshold:", vars_to_drop.tolist())

# Drop only those identified variables from df_dropped
df_dropped = df_dropped.drop(columns=vars_to_drop)
loadings

Variance explained by each latent variable in PCA:  [0.47620343 0.27199895 0.11307971] [0.47620343 0.74820238 0.86128209] in 0.1111111111111111 threshold for loadings
Variables to be dropped based on loadings threshold: ['miw_new', 'banzhaf', 'shapley', 'splus', 'party_count', 'cab_count', 'enpp', 'miw_proportion', 'W']


,miw_new,banzhaf,shapley,splus,party_count,cab_count,enpp,miw_proportion,W
PC1,0.006302,0.459681,0.462471,0.460956,0.254834,0.180730,0.201702,0.465262,0.086824
PC2,0.500781,0.171032,0.171958,0.170710,0.477333,0.437533,0.345182,0.126240,0.326933
PC3,0.058425,0.054769,0.051265,0.049668,0.055472,0.023710,0.625461,0.013498,0.770369


In [332]:
# -------------------------------------------------------Training-------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor 

# Data preparation
Y = df_dropped['cabinet_proportion']
X = df_dropped.drop(columns=['cabinet_proportion'])
X = pd.concat([X, new_df, new_df2], axis=1)  # Merge all latent variables

# Save the dataset to a CSV file
output_path = '/Users/jz/Desktop/ML Final/output.csv'
pd.concat([Y, X], axis=1).to_csv(output_path, index=False)
print(f"Data saved to {output_path}")


Data saved to /Users/jz/Desktop/ML Final/output.csv


In [334]:
# Define the models to test with scaling included
model_configs = [
    {
        'type': 'lasso_linear', 
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('lasso', LassoCV(
                alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], 
                max_iter=50000,    # Increased max_iter for better convergence
                tol=1e-3,          # Adjusted tolerance
                cv=5, 
                random_state=42
            ))
        ])
    },
    {
        'type': 'lasso_polynomial', 
        'model': Pipeline([
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler()),
            ('lasso', LassoCV(
                alphas=[0.01, 0.1, 1, 10], 
                max_iter=50000,    # Increased max_iter for better convergence
                tol=1e-3,          # Adjusted tolerance
                cv=5, 
                random_state=42
            ))
        ])
    },
    {
        'type': 'gradient_boosting',
        'model': GradientBoostingRegressor(random_state=42)
    },
    {
        'type': 'decision_tree',  # New Decision Tree configuration
        'model': DecisionTreeRegressor(random_state=42)
    }
]

# Define the parameter grids
gb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Parameter grid for Decision Tree
dt_param_grid = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Initialize a dictionary to store the best models for each type
best_models = {}

# Define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Iterate through each model configuration
for config in model_configs:
    model_type = config['type']
    model = config['model']
    
    print(f"\nProcessing Model Type: {model_type.replace('_', ' ').capitalize()}")
    
    if model_type == 'gradient_boosting':
        # Perform Grid Search for Gradient Boosting
        grid_search = GridSearchCV(
            estimator=model, 
            param_grid=gb_param_grid, 
            scoring='neg_mean_squared_error', # this is the cv MSE
            cv=cv, 
            verbose=1,
            n_jobs=-1
        )
        grid_search.fit(X, Y)
        best_estimator = grid_search.best_estimator_
        best_params = grid_search.best_params_
        
        # Cross-validated MSE
        neg_mse = cross_val_score(
            best_estimator, X, Y, 
            cv=cv, 
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        mse = -neg_mse.mean()
        
        # Store the best model details
        best_models[model_type] = {
            'model': best_estimator,
            'params': best_params,
            'mse': mse,
            'alpha': None  # No alpha for Gradient Boosting
        }
        print(f"Best Parameters: {best_params}")
        print(f"Cross-Validated MSE: {mse:.4f}")
        
    elif model_type == 'decision_tree':  # Handling Decision Tree
        # Perform Grid Search for Decision Tree
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=dt_param_grid,
            scoring='neg_mean_squared_error', # this is the cv MSE
            cv=cv,
            verbose=1,
            n_jobs=-1
        )
        grid_search.fit(X, Y)
        best_estimator = grid_search.best_estimator_
        best_params = grid_search.best_params_
        
        # Cross-validated MSE
        neg_mse = cross_val_score(
            best_estimator, X, Y,
            cv=cv,
            scoring='neg_mean_squared_error', # this is the cv MSE
            n_jobs=-1
        )
        mse = -neg_mse.mean()
        
        # Store the best model details
        best_models[model_type] = {
            'model': best_estimator,
            'params': best_params,
            'mse': mse,
            'alpha': None  # No alpha for Decision Tree
        }
        print(f"Best Parameters: {best_params}")
        print(f"Cross-Validated MSE: {mse:.4f}")
        
    elif 'lasso' in model_type:
        # Fit the Lasso model using cross-validation to find the best alpha
        model.fit(X, Y)
        best_alpha = model.named_steps['lasso'].alpha_
        
        # Cross-validated MSE
        neg_mse = cross_val_score(
            model, X, Y, 
            cv=cv, 
            scoring='neg_mean_squared_error', # this is the cv MSE
            n_jobs=-1
        )
        mse = -neg_mse.mean()
        
        # Store the best model details
        best_models[model_type] = {
            'model': model,
            'params': {'alpha': best_alpha},
            'mse': mse,
            'alpha': best_alpha
        }
        print(f"Best Alpha: {best_alpha}")
        print(f"Cross-Validated MSE: {mse:.4f}")
        
    else:
        # For Linear and Polynomial models
        # Cross-validated MSE
        neg_mse = cross_val_score(
            model, X, Y, 
            cv=cv, 
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        mse = -neg_mse.mean()
        
        # Fit the model on the entire dataset
        model.fit(X, Y)
        
        # Store the best model details
        best_models[model_type] = {
            'model': model,
            'params': {},
            'mse': mse,
            'alpha': None
        }
        print(f"Cross-Validated MSE: {mse:.4f}")

# Output Best Models Details
print("\n\n================================ Best Models Summary =================================\n")
for model_type, details in best_models.items():
    print(f"Model Type: {model_type.replace('_', ' ').capitalize()}")
    print(f"Cross-Validated MSE: {details['mse']:.4f}")
    if details['params']:
        print(f"Parameters: {details['params']}")
    else:
        print("Parameters: Default")
    
    if model_type in ['gradient_boosting', 'decision_tree']:
        # Feature Importance for Tree-based models
        feature_importance = details['model'].feature_importances_
        feature_names = X.columns
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importance
        })
        # Sort by absolute importance
        importance_df['Abs_Importance'] = importance_df['Importance'].abs()
        importance_df = importance_df.sort_values(by='Abs_Importance', ascending=False).drop(columns=['Abs_Importance'])
        print("\nFeature Importances (sorted by absolute value):")
        print(importance_df)
    
    elif 'lasso' in model_type:
        # Coefficients for Lasso models
        if 'polynomial' in model_type:
            poly = details['model'].named_steps['poly']
            feature_names = poly.get_feature_names_out(X.columns)
        else:
            feature_names = X.columns
        coefficients = details['model'].named_steps['lasso'].coef_
        intercept = details['model'].named_steps['lasso'].intercept_
        coef_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': coefficients
        })
        # Sort by absolute coefficient value
        coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
        coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False).drop(columns=['Abs_Coefficient'])
        print(f"\nIntercept: {intercept:.4f}")
        print("Coefficients (sorted by absolute value):")
        print(coef_df)
    
    else:
        # Coefficients for Linear and Polynomial models
        if model_type == 'polynomial':
            poly = details['model'].named_steps['poly']
            feature_names = poly.get_feature_names_out(X.columns)
        else:
            feature_names = X.columns
        coefficients = details['model'].named_steps['linear'].coef_
        intercept = details['model'].named_steps['linear'].intercept_
        coef_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': coefficients
        })
        # Sort by absolute coefficient value
        coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
        coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False).drop(columns=['Abs_Coefficient'])
        print(f"\nIntercept: {intercept:.4f}")
        print("Coefficients (sorted by absolute value):")
        print(coef_df)
    
    print("\n---------------------------------------------------------------------------------------\n")



Processing Model Type: Lasso linear
Best Alpha: 0.001
Cross-Validated MSE: 0.0068

Processing Model Type: Lasso polynomial
Best Alpha: 0.01
Cross-Validated MSE: 0.0022

Processing Model Type: Gradient boosting
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Cross-Validated MSE: 0.0018

Processing Model Type: Decision tree
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}
Cross-Validated MSE: 0.0078


================================ Best Models Summary =================================

Model Type: Lasso linear
Cross-Validated MSE: 0.0068
Parameters: {'alpha': 0.001}

Intercept: 0.1300
Coefficients (sorted by absolute value):
                                              Feature  Coefficient
4                                       cabinet_party     0.099130
29 

In [319]:
# save the best model summary to CSV

summary = []
for model_type, details in best_models.items():
    entry = {
        'Model Type': model_type,
        'MSE': details['mse'],
        'Hyperparameters': details['params']
    }
    summary.append(entry)
summary_df = pd.DataFrame(summary)
summary_df.to_csv('/Users/jz/Desktop/ML Final/best_models_summary.csv', index=False)
print("Best models summary saved to CSV.")

Best models summary saved to CSV.
